In [1]:
import pandas as pd

df = pd.read_csv('../raw_data/data.csv')[:100_000]

/Users/alicedebokay/.pyenv/versions/3.8.6/envs/minority_report/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df.shape

(100000, 22)

In [3]:
def drop_nan(df):
    '''
    Returns a dataframe without NaN
    '''
    df = df.copy()
    #drop precinct missing values
    not_unknown = df['precinct_number'] != -99.0
    not_nan = df['precinct_number'] == df['precinct_number']
    df = df[not_unknown & not_nan]
    #drop time and date missing values
    df = df[df['time'] == df['time']]
    df = df[df['date'] == df['date']]
    #drop offense_type and crime_completed
    df = df[df['offense_type'] == df['offense_type']]
    df = df[df['crime_completed'] == df['crime_completed']]
    return df

In [4]:
df = drop_nan(df)

In [5]:
df["period"] = df["date"] + ' ' + df["time"]

In [6]:
df['period']

0        02/11/2015 15:00:00
1        03/17/2012 10:30:00
2        10/27/2016 13:48:00
3        11/27/2014 19:00:00
4        12/11/2013 13:30:00
                ...         
99995    09/13/2007 09:50:00
99996    12/14/2011 21:30:00
99997    11/27/2011 02:00:00
99998    12/05/2007 15:25:00
99999    03/28/2011 23:30:00
Name: period, Length: 99595, dtype: object

In [ ]:
import pandas
import datetime
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))
x = pandas.read_csv('data.csv',delimiter=';', parse_dates=True,date_parser=dateparse, index_col='DateTime', names=['DateTime', 'X'], header=None)
out = x.truncate(before=datetime.datetime(2015,12,2,12,2,18))

In [26]:
from datetime import datetime

def to_date_format(df):
    '''
    Returns dataframe with 'date' as datetime dtype
    Filters dataframe to show only complaints dated 2007 onwards
    '''
    df = df.copy()
    df['period'] = df['period'].apply(lambda x: \
                                datetime.strptime(x, '%m/%d/%Y %H:%M:%S'))
    df['period'] = df['period'].apply(lambda x: rounder(x))
    df = df[df['period'] > datetime(2006, 12, 31, 23, 59, 0)]
    return df

In [27]:
def rounder(t):
    '''Rounds all hours to closest hour.'''
    if t.minute >= 30:
        return t.replace(hour = t.hour + 1, minute = 0, second = 0, microsecond = 0)
    else:
        return t.replace(minute = 0, second = 0, microsecond = 0)

In [28]:
df = to_date_format(df)

TypeError: strptime() argument 1 must be str, not Timestamp

In [15]:
df['period'][2]

Timestamp('2016-10-27 13:48:00')

In [10]:
type(df['period'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [13]:
df['year'] = df['period'].apply(lambda x: x.year)

In [14]:
df.head(3)

,complaint_id,date,time,precinct_number,offense_type,crime_completed,offense_level,borough,premise_desc,premise,...,suspect_sex,latitude,longitude,patrol_borough,metro,victim_age,victim_race,victim_sex,period,year
0,325341655,02/11/2015,15:00:00,73.0,OFFENSES AGAINST PUBLIC ADMINI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,...,M,40.664239,-73.908425,PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M,2015-02-11 15:00:00,2015
1,393816841,03/17/2012,10:30:00,69.0,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,...,NaN,40.644590,-73.892672,PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F,2012-03-17 10:30:00,2012
2,802896158,10/27/2016,13:48:00,71.0,HARRASSMENT 2,COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,...,M,40.658758,-73.942435,PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M,2016-10-27 13:48:00,2016


In [25]:
df.drop(columns = ['date', 'time'])

,complaint_id,precinct_number,offense_type,crime_completed,offense_level,borough,premise_desc,premise,jurisdiction,park_name,...,suspect_sex,latitude,longitude,patrol_borough,metro,victim_age,victim_race,victim_sex,period,year
0,325341655,73.0,OFFENSES AGAINST PUBLIC ADMINI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,NaN,...,M,40.664239,-73.908425,PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M,2015-02-11 15:00:00,2015
1,393816841,69.0,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,NaN,...,NaN,40.644590,-73.892672,PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F,2012-03-17 10:30:00,2012
2,802896158,71.0,HARRASSMENT 2,COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,NaN,...,M,40.658758,-73.942435,PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M,2016-10-27 13:48:00,2016
3,633812343,112.0,RAPE,COMPLETED,FELONY,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NaN,...,M,40.722364,-73.851474,PATROL BORO QUEENS NORTH,NaN,25-44,WHITE,F,2014-11-27 19:00:00,2014
4,300349533,24.0,GRAND LARCENY,COMPLETED,FELONY,MANHATTAN,INSIDE,DOCTOR/DENTIST OFFICE,N.Y. POLICE DEPT,NaN,...,NaN,40.793465,-73.968950,PATROL BORO MAN NORTH,NaN,45-64,WHITE,F,2013-12-11 13:30:00,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,499648605,61.0,CRIMINAL MISCHIEF & RELATED OF,COMPLETED,MISDEMEANOR,BROOKLYN,FRONT OF,RESIDENCE-HOUSE,N.Y. POLICE DEPT,NaN,...,NaN,40.587058,-73.936448,PATROL BORO BKLYN SOUTH,NaN,NaN,UNKNOWN,E,2007-09-13 09:50:00,2007
99996,261360313,52.0,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,BRONX,FRONT OF,STREET,N.Y. POLICE DEPT,NaN,...,NaN,40.861843,-73.902951,PATROL BORO BRONX,NaN,NaN,UNKNOWN,M,2011-12-14 21:30:00,2011
99997,205495976,114.0,CRIMINAL MISCHIEF & RELATED OF,COMPLETED,FELONY,QUEENS,NaN,STREET,N.Y. POLICE DEPT,NaN,...,NaN,40.780131,-73.911526,PATROL BORO QUEENS NORTH,NaN,<18,WHITE HISPANIC,F,2011-11-27 02:00:00,2011
99998,712555932,52.0,PETIT LARCENY,COMPLETED,MISDEMEANOR,BRONX,FRONT OF,STREET,N.Y. POLICE DEPT,NaN,...,M,40.877927,-73.882373,PATROL BORO BRONX,NaN,NaN,WHITE,F,2007-12-05 15:25:00,2007


In [20]:
import pickle

In [21]:
with open('test.pickle', 'wb') as f:
    pickle.dump(df.head(3), f)

In [22]:
with open('test.pickle', 'rb') as f:
    data_test = pickle.load(f)

In [24]:
data_test['period'][0]

Timestamp('2015-02-11 15:00:00')

In [16]:
data = pd.read_csv('../raw_data/clean.csv')

In [17]:
data.head(3)

,complaint_id,date,time,precinct_number,offense_type,crime_completed,offense_level,premise_desc,premise,jurisdiction,...,suspect_sex,latitude,longitude,metro,victim_age,victim_race,victim_sex,period,borough,patrol_borough
0,325341655,02/11/2015,15:00:00,73,OFFENSES AGAINST PUBLIC ADMINI,True,MISDEMEANOR,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,...,M,40.664239,-73.908425,NOT SUBWAY,<18,BLACK,M,2015-02-11 15:00:00,BROOKLYN,PATROL BORO BKLYN NORTH
1,393816841,03/17/2012,10:30:00,69,ASSAULT 3 & RELATED OFFENSES,True,MISDEMEANOR,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,...,UNKNOWN,40.644590,-73.892672,NOT SUBWAY,45-64,BLACK,F,2012-03-17 10:30:00,BROOKLYN,PATROL BORO BKLYN SOUTH
2,802896158,10/27/2016,13:48:00,71,HARRASSMENT 2,True,VIOLATION,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,...,M,40.658758,-73.942435,NOT SUBWAY,18-24,BLACK,M,2016-10-27 13:48:00,BROOKLYN,PATROL BORO BKLYN SOUTH


In [18]:
type(data['period'][2])

str

In [19]:
data['period'][2]

'2016-10-27 13:48:00'